# Notebook for the main analysis of the annotations 
Here, the amount of each signal type (pulse, sine, vibration) is analyzed. The results are exported for plotting and statistical testing. The analysis is based on raster plots which are created from annotations that are aligned to the stimulation onset. 

Author: Bjarne Schultze <br>
Last modified: 29.11.2024

In [1]:
# Necessary modules import
import numpy as np
import pandas as pd
import itertools
import h5py

import modules.analysis_utils as utils
import modules.data_handling_utils as dutils

#### Options for running the analysis
These values can be changed to modify the analysis.

In [3]:
# Set whether or not to export data for statistical testing
export_test_data = True

# Choose an experimental group
exp_grp = "vMS12-SS3" 

# Set the padding around the stimulation period which should be included in the analysis
pre_stim_pad = 5    # [s]
post_stim_pad = 5   # [s]

# Set a bin width
bin_width = 0.1    # [s]

#### Preparations

Load the metadata, the light intensity values, and the annotations.

In [4]:
# Create a dictionary encoding signal types as numbers
type_dict = {0:"None", 1:"Pulse", 2:"Sine", 3:"Vib"}
# Create a dictionary mapping experimental group and genotype in the meta data table
genotype_dict = {"pIP10": "CsChrimson; VT40556", "TN1A": "TN1A_CsChrimson", "vPR13": "vPR13_CsChrimson", 
                 "vMS12-SS3": "vMS12-SS3_CsChrimson", "CsChrimson_ctrl": "CsChrimson", "vPR13_ctrl": "vPR13"}

# Set the main result path
main_path = "E:/res/"
# Set the path to the metadata file
metadata_path = "./accessory_files/metadata.pkl"

# Load metadata file (first five files with different protocol are missing)
metadata = pd.read_pickle(metadata_path)

# Extract the experiment names for the current neuron type
exp = metadata.loc[metadata["genotype"] == genotype_dict[exp_grp], :]
experiments = exp.loc[exp["individuals.1"].isna(), "filename"].to_list()
experiments_mf = exp.loc[exp["individuals.1"] == "1", "filename"].to_list()

# Read the calibration table for the light intensity
if exp_grp == "pIP10":    # For the pIP10 protocol
    with open("./accessory_files/opto_calibration_120_5.txt") as f:
        lines = f.readlines()
    opto_calib = np.sort(np.array([ txt.removesuffix('\n').split(',') for txt in lines ]).astype('float'), axis=0)
else:       # For the normal protocol (all other conditions)
    with open("./accessory_files/opto_calibration_26_4.txt") as f:
        lines = f.readlines()
    opto_calib = np.sort(np.array([ txt.removesuffix('\n').split(',') for txt in lines ]).astype('float'), axis=0)

In [4]:
# Load the annotations for the male-female and solitary condition
annotations_mf, stim_ons_mf, stim_offs_mf, stim_volts_mf, _ = dutils.load_annotations(experiments_mf, main_path=main_path, metadata_path=metadata_path)
annotations, stim_ons, stim_offs, stim_volts, _ = dutils.load_annotations(experiments, main_path=main_path, metadata_path=metadata_path)

# Check if there are data for both conditions (sm: single male, mf: male-female)
if len(annotations) > 0: sm_cond = True
else: sm_cond = False
if len(annotations_mf) > 0: mf_cond = True
else: mf_cond = False

# Calculate the stimulus length
stim_len = stim_offs[0][0] - stim_ons[0][0]

# Extract the sampling rate
sampling_rate = metadata["sampling_rate"][5]

#### Analysis
 
Create raster matrices from the annotations. Find trains for each signal type (pulse, sine, vibration). Find potential overlaps between signal types. 
Create raster matrices suitable for plotting. Count the signal type changes.

In [5]:
if mf_cond:
    # Create raster matrices with fine bin size from the annotations (male-female condition)
    full_raster_p_mf_fine, raster_edges_mf_fine = utils.psth(annotations_mf, stim_ons_mf, stim_len, event_label='pulse_manual', bin_width=1/sampling_rate, output='full')
    full_raster_v_mf_fine, _ = utils.psth(annotations_mf, stim_ons_mf, stim_len, event_label='vibration_manual', bin_width=1/sampling_rate, output='full')
    full_raster_s_mf_fine, _ = utils.psth(annotations_mf, stim_ons_mf, stim_len, event_label='sine_manual', bin_width=1/sampling_rate, output='full')
if sm_cond:
    # For the solitary male condition
    full_raster_p_fine, raster_edges_fine = utils.psth(annotations, stim_ons, stim_len, event_label='pulse_manual', bin_width=1/sampling_rate, output='full')
    full_raster_v_fine, _ = utils.psth(annotations, stim_ons, stim_len, event_label='vibration_manual', bin_width=1/sampling_rate, output='full')
    full_raster_s_fine, _ = utils.psth(annotations, stim_ons, stim_len, event_label='sine_manual', bin_width=1/sampling_rate, output='full')

In [6]:
# Allocate lists to collect the potential overlap between vibration and pulse/sine
vp_overlap = []
vs_overlap = []
vp_overlap_mf = []
vs_overlap_mf = []

## Find trains of a signal type
if sm_cond:
    # For pulse
    ptrain_start, ptrain_stop, ptrain_len = utils.find_evt_trains_multi(full_raster_p_fine, raster_edges_fine[:-1], 60)
    # For sine
    strain_start, strain_stop, strain_len = utils.find_evt_trains_multi(full_raster_s_fine, raster_edges_fine[:-1], 1.1)
    # For vibration
    vtrain_start, vtrain_stop, vtrain_len = utils.find_evt_trains_multi(full_raster_v_fine, raster_edges_fine[:-1], 300)

    # Iterate over the trials (rows of the full raster matrix)
    for i in range(full_raster_p_fine.shape[0]):
        if ptrain_start[i].shape[0] > 0 and vtrain_start[i].shape[0] > 0:
            # Find potential overlaps between the signal types
            vp_overlap.append(utils.overlap(ptrain_start[i], ptrain_stop[i], vtrain_start[i], vtrain_stop[i]))
        else:
            vp_overlap.append(np.array([]))

        if strain_start[i].shape[0] > 0 and vtrain_start[i].shape[0] > 0:
            # Find potential overlaps between the signal types
            vs_overlap.append(utils.overlap(strain_start[i], strain_stop[i], vtrain_start[i], vtrain_stop[i]))
        else:
            vs_overlap.append(np.array([]))


    ## Filter out trials with no overlaps
    # For the pulse-vibration overlaps
    # For the solitary male condition
    overlaps_vp = []
    overlaps_vp_idx = []
    _ = [ (overlaps_vp_idx.append(np.repeat(vp, vp_overlap[vp].shape[0])), overlaps_vp.append(vp_overlap[vp])) 
        for vp in range(len(vp_overlap)) if vp_overlap[vp].shape[0] > 0 ]
    # Convert to numpy arrays
    if len(overlaps_vp) > 0:
        overlaps_vp = np.vstack(overlaps_vp)
        overlaps_vp_idx = np.concatenate(overlaps_vp_idx)

    # For the sine-vibration overlaps
    overlaps_vs = []
    overlaps_vs_idx = []
    _ = [ (overlaps_vs_idx.append(np.repeat(vs, vs_overlap[vs].shape[0])), overlaps_vs.append(vs_overlap[vs])) 
        for vs in range(len(vs_overlap)) if vs_overlap[vs].shape[0] > 0 ]
    # Convert to numpy arrays
    if len(overlaps_vs) > 0: 
        overlaps_vs = np.vstack(overlaps_vs)
        overlaps_vs_idx = np.concatenate(overlaps_vs_idx)

if mf_cond:
    # For the male-female condition
    ptrain_start_mf, ptrain_stop_mf, ptrain_len_mf = utils.find_evt_trains_multi(full_raster_p_mf_fine, raster_edges_mf_fine[:-1], 60)
    strain_start_mf, strain_stop_mf, strain_len_mf = utils.find_evt_trains_multi(full_raster_s_mf_fine, raster_edges_mf_fine[:-1], 1.1)
    vtrain_start_mf, vtrain_stop_mf, vtrain_len_mf = utils.find_evt_trains_multi(full_raster_v_mf_fine, raster_edges_mf_fine[:-1], 300)

    # Iterate over the trials (rows of the full raster matrix)
    for i in range(full_raster_p_mf_fine.shape[0]):
        if ptrain_start_mf[i].shape[0] > 0 and vtrain_start_mf[i].shape[0] > 0:
            # Find potential overlaps between the signal types
            vp_overlap_mf.append(utils.overlap(ptrain_start_mf[i], ptrain_stop_mf[i], vtrain_start_mf[i], vtrain_stop_mf[i]))
        else:
            vp_overlap_mf.append(np.array([]))

        if strain_start_mf[i].shape[0] > 0 and vtrain_start_mf[i].shape[0] > 0:
            # Find potential overlaps between the signal types
            vs_overlap_mf.append(utils.overlap(strain_start_mf[i], strain_stop_mf[i], vtrain_start_mf[i], vtrain_stop_mf[i]))
        else:
            vs_overlap_mf.append(np.array([]))


    ## Filter out trials with no overlaps
    # For the pulse-vibration overlaps
    overlaps_vp_mf = []
    overlaps_vp_mf_idx = []
    _ = [ (overlaps_vp_mf_idx.append(np.repeat(vp, vp_overlap_mf[vp].shape[0])), overlaps_vp_mf.append(vp_overlap_mf[vp])) 
        for vp in range(len(vp_overlap_mf)) if vp_overlap_mf[vp].shape[0] > 0 ]
    # Convert to numpy arrays
    if len(overlaps_vp_mf) > 0:
        overlaps_vp_mf = np.vstack(overlaps_vp_mf)
        overlaps_vp_mf_idx = np.concatenate(overlaps_vp_mf_idx)

    # For the sine-vibration overlaps
    overlaps_vs_mf = []
    overlaps_vs_mf_idx = []
    _ = [ (overlaps_vs_mf_idx.append(np.repeat(vs, vs_overlap_mf[vs].shape[0])), overlaps_vs_mf.append(vs_overlap_mf[vs])) 
        for vs in range(len(vs_overlap_mf)) if vs_overlap_mf[vs].shape[0] > 0 ]
    # Convert to numpy arrays
    if len(overlaps_vs_mf) > 0:
        overlaps_vs_mf = np.vstack(overlaps_vs_mf)
        overlaps_vs_mf_idx = np.concatenate(overlaps_vs_mf_idx)

In [7]:
# Get time vector for the raster matrices
raster_time = np.arange(-pre_stim_pad, stim_len+post_stim_pad+bin_width, bin_width)[:-1]

# Create an array with the possible signal type changes
poss_evt_chgs = np.unique([i for i in itertools.permutations([1.0, 2.0, 3.0, 0.0, 1.0, 2.0, 3.0], 2) ], axis=0)

if mf_cond:
    # Get time vector 
    raster_time_fine = raster_edges_mf_fine[:-1]
    # Fill raster matrices using the event train information (fine bin width)
    raster_mat_p_mf = utils.raster_mat_from_trains(ptrain_start_mf, ptrain_stop_mf, raster_time_fine, full_raster_p_mf_fine.shape, fill=1)
    raster_mat_s_mf = utils.raster_mat_from_trains(strain_start_mf, strain_stop_mf, raster_time_fine, full_raster_s_mf_fine.shape, fill=2)
    raster_mat_v_mf = utils.raster_mat_from_trains(vtrain_start_mf, vtrain_stop_mf, raster_time_fine, full_raster_v_mf_fine.shape, fill=3)

    # Make a version with larger bin size for plotting
    r_mat_mf_shape = (full_raster_p_mf_fine.shape[0], raster_time.shape[0])
    r_mat_p_mf = utils.raster_mat_from_trains(ptrain_start_mf, ptrain_stop_mf, raster_time, r_mat_mf_shape, fill=1)
    r_mat_s_mf = utils.raster_mat_from_trains(strain_start_mf, strain_stop_mf, raster_time, r_mat_mf_shape, fill=2)
    r_mat_v_mf = utils.raster_mat_from_trains(vtrain_start_mf, vtrain_stop_mf, raster_time, r_mat_mf_shape, fill=3)

    # Get all signal type changes in the male and male-female condition
    changes_mf, chg_idx_mf = utils.get_signal_switches([raster_mat_p_mf,raster_mat_s_mf,raster_mat_v_mf], 1/sampling_rate, tolerance=0.1)
    # Get the trial numbers for all experiments
    exp_trial_num_mf = [ len(exp) for exp in stim_volts_mf ]
    # Calculate the number of signal changes (male+female)
    cnt_changes_pre_norm_mf, cnt_changes_dur_norm_mf = utils.count_signal_switches_exp_wise([raster_mat_p_mf, raster_mat_s_mf, raster_mat_v_mf],
                                                                                            raster_time_fine, poss_evt_chgs, exp_trial_num_mf,
                                                                                            bin_width=1/sampling_rate, tolerance=0.1, 
                                                                                            stim_len=stim_len)

if sm_cond:
    ## Repeat for the solitary male data
    # Get time vector 
    raster_time_fine = raster_edges_fine[:-1]
    # Fill raster matrices using the event trains
    raster_mat_p = utils.raster_mat_from_trains(ptrain_start, ptrain_stop, raster_time_fine, full_raster_p_fine.shape, fill=1)
    raster_mat_s = utils.raster_mat_from_trains(strain_start, strain_stop, raster_time_fine, full_raster_s_fine.shape, fill=2)
    raster_mat_v = utils.raster_mat_from_trains(vtrain_start, vtrain_stop, raster_time_fine, full_raster_v_fine.shape, fill=3)

    # Make a version with larger bin size for plotting
    r_mat_shape = (full_raster_p_fine.shape[0], raster_time.shape[0])
    r_mat_p = utils.raster_mat_from_trains(ptrain_start, ptrain_stop, raster_time, r_mat_shape, fill=1)
    r_mat_s = utils.raster_mat_from_trains(strain_start, strain_stop, raster_time, r_mat_shape, fill=2)
    r_mat_v = utils.raster_mat_from_trains(vtrain_start, vtrain_stop, raster_time, r_mat_shape, fill=3)


    # Get all signal type changes in the male and male-female condition
    changes, chg_idx = utils.get_signal_switches([raster_mat_p,raster_mat_s,raster_mat_v], 1/sampling_rate, tolerance=0.1)
    # Get the trial numbers for all experiments
    exp_trial_num = [ len(exp) for exp in stim_volts ]
    # Calculate the number of signal changes (male)
    cnt_changes_pre_norm, cnt_changes_dur_norm = utils.count_signal_switches_exp_wise([raster_mat_p, raster_mat_s, raster_mat_v],
                                                                                    raster_time_fine, poss_evt_chgs, exp_trial_num,
                                                                                    bin_width=1/sampling_rate, tolerance=0.1, 
                                                                                    stim_len=stim_len)

In [8]:
# Create an index for saving the data later
global_changes_idx = ["pulse","sine","vib"]

if mf_cond:
    # Quantify the song modes and vibrations in FI curves
    p_counts_grp_mf, vib_counts_grp_mf, sine_counts_grp_mf = utils.annotation_fi_curve(annotations_mf, stim_ons_mf, stim_offs_mf, stim_volts_mf)
    # Combine all data in one array
    all_counts_mf = np.vstack([p_counts_grp_mf, sine_counts_grp_mf, vib_counts_grp_mf])
    # Create indices 
    signal_index = np.concatenate([ np.repeat(m, len(p_counts_grp_mf)) for m in zip(["pulse","sine","vib"])])
    stim_set = np.unique(np.concatenate(stim_volts_mf))
    stim_index = np.concatenate([stim_set, stim_set, stim_set])

    ## Calculate the same information but averaged across all intensities for each fly
    # Create fake stimulus volt lists (same intensity for all entries)
    fake_stim_volts_mf = [ np.ones(stv.shape[0]) for stv in stim_volts_mf ]
    # Calculate the counts averaged over all intensities
    p_counts_idvsum_mf, vib_counts_idvsum_mf, sine_counts_idvsum_mf = utils.annotation_fi_curve(annotations_mf, stim_ons_mf, stim_offs_mf, fake_stim_volts_mf)

    # Combine averaged counts in one array
    global_signal_changes_mf = np.vstack([p_counts_idvsum_mf[0], sine_counts_idvsum_mf[0], vib_counts_idvsum_mf[0]])


    # Export the data for testing in R if requested
    if export_test_data:
        # Create a array with all stimulus intensities
        stim_intensity_mf = np.concatenate([ np.repeat(opto_calib[:,1][i], p_counts_grp_mf[i].shape[0]) for i in range(len(p_counts_grp_mf)) ])
        individual_idx_mf = np.concatenate([ list(range(p_counts_grp_mf[i].shape[0])) for i in range(len(p_counts_grp_mf)) ])
        data_mf = pd.DataFrame({"ID":individual_idx_mf, "stim":stim_intensity_mf, "pulse":np.concatenate(p_counts_grp_mf), 
                            "sine":np.concatenate(sine_counts_grp_mf), "vib":np.concatenate(vib_counts_grp_mf)})
        # Store data as csv
        data_mf.to_csv(f"../additional_files/{exp_grp}_changes_to_test_mf.csv")
else:
    # Create empty if no data to avoid "possibly unbound" errors 
    p_counts_idvsum_mf = [np.array([])]
    sine_counts_idvsum_mf = [np.array([])]
    vib_counts_idvsum_mf = [np.array([])]

if sm_cond:
    # Quantify the song modes and vibrations in FI curves
    p_counts_grp, vib_counts_grp, sine_counts_grp = utils.annotation_fi_curve(annotations, stim_ons, stim_offs, stim_volts)
    # Combine all data in one array
    all_counts = np.vstack([p_counts_grp, sine_counts_grp, vib_counts_grp])
    # Create indices 
    signal_index = np.concatenate([ np.repeat(m, len(p_counts_grp)) for m in zip(["pulse","sine","vib"])])
    stim_set = np.unique(np.concatenate(stim_volts))
    stim_index = np.concatenate([stim_set, stim_set, stim_set])

    ## Calculate the same information but averaged across all intensities for each fly
    # Create fake stimulus volt lists (same intensity for all entries)
    fake_stim_volts = [ np.ones(stv.shape[0]) for stv in stim_volts ]
    # Calculate the counts averaged over all intensities
    p_counts_idvsum, vib_counts_idvsum, sine_counts_idvsum = utils.annotation_fi_curve(annotations, stim_ons, stim_offs, fake_stim_volts)

    # Combine averaged counts in one array
    global_signal_changes = np.vstack([p_counts_idvsum[0], sine_counts_idvsum[0], vib_counts_idvsum[0]])
    # Create an index
    global_changes_idx = ["pulse","sine","vib"] 


    # Export the data for testing in R if requested
    if export_test_data:
        # Create a array with all stimulus intensities
        stim_intensity_m = np.concatenate([ np.repeat(opto_calib[:,1][i], p_counts_grp[i].shape[0]) for i in range(len(p_counts_grp)) ])
        # Index array resolving which data point came from which animal
        individual_idx_m = np.concatenate([ list(range(p_counts_grp[i].shape[0])) for i in range(len(p_counts_grp)) ])
        # Combine data in a data frame
        data_m = pd.DataFrame({"ID":individual_idx_m, "stim":stim_intensity_m, "pulse":np.concatenate(p_counts_grp), 
                            "sine":np.concatenate(sine_counts_grp), "vib":np.concatenate(vib_counts_grp)})
        # Store data as csv
        data_m.to_csv(f"../additional_files/{exp_grp}_changes_to_test_m.csv")
else:
    # Create empty if no data
    p_counts_idvsum = [np.array([])]
    sine_counts_idvsum = [np.array([])]
    vib_counts_idvsum = [np.array([])]


# Export the data for testing in R if requested
if export_test_data:
    # Array resolving the condition (m/mf) for each data point of the data averaged across all stimulus intensities
    condition = np.concatenate([np.repeat("m", p_counts_idvsum[0].shape[0]), np.repeat("mf", p_counts_idvsum_mf[0].shape[0])])
    # Combine data in a data frame
    data_test = pd.DataFrame({"condition":condition, 
                            "pulse":np.concatenate([p_counts_idvsum[0], p_counts_idvsum_mf[0]]), 
                            "sine":np.concatenate([sine_counts_idvsum[0], sine_counts_idvsum_mf[0]]), 
                            "vib":np.concatenate([vib_counts_idvsum[0], vib_counts_idvsum_mf[0]])})
        
    # Store the data as csv
    data_test.to_csv(f"../additional_files/{exp_grp}_changes_to_test.csv")

In [9]:
# Sort the labels according to the second entry (type after switch)
srt_idx_chgs = np.argsort(poss_evt_chgs[:,1], axis=0)
labels_srt = poss_evt_chgs[srt_idx_chgs,:]

# Lists to combine the data from prior and during stimulation
cnt_chg_combi_mf = []
cnt_chg_combi = []
combi_labels = []

# Create a custom sort index (leaves out the "no-changes")
srt_idx_combi = np.array([0,1,16,17,24,25, 2,3,10,11,26,27, 4,5,12,13,20,21, 6,7,14,15,22,23])

if mf_cond:
    # Sort the switch counts accordingly
    cnt_chgs_dur_srt_mf = cnt_changes_dur_norm_mf[:, srt_idx_chgs]
    cnt_chgs_pre_srt_mf = cnt_changes_pre_norm_mf[:, srt_idx_chgs]

    # Combine counts and labels
    [ cnt_chg_combi_mf.extend([cnt_chgs_pre_srt_mf[:,i], cnt_chgs_dur_srt_mf[:,i]]) for i in range(labels_srt.shape[0]) ]
    [ combi_labels.extend([poss_evt_chgs[i,:], poss_evt_chgs[i,:]]) for i in range(labels_srt.shape[0]) ]
    # Convert to numpy arrays
    cnt_chg_combi_mf = np.vstack(cnt_chg_combi_mf).T

    # Apply index to counts
    cnt_chg_combi_srt_mf = cnt_chg_combi_mf[:, srt_idx_combi]
else:
    # Create empty if no data
    cnt_chgs_pre_srt_mf = np.repeat(np.nan, labels_srt.shape[0]).reshape((1, labels_srt.shape[0]))
    cnt_chgs_dur_srt_mf = np.repeat(np.nan, labels_srt.shape[0]).reshape((1, labels_srt.shape[0]))

if sm_cond:
    # Repeat for the solitary male
    cnt_chgs_pre_srt = cnt_changes_pre_norm[:, srt_idx_chgs]
    cnt_chgs_dur_srt = cnt_changes_dur_norm[:, srt_idx_chgs]

    # Combine counts and labels
    [ cnt_chg_combi.extend([cnt_chgs_pre_srt[:,i], cnt_chgs_dur_srt[:,i]]) for i in range(labels_srt.shape[0]) ]
    [ combi_labels.extend([poss_evt_chgs[i,:], poss_evt_chgs[i,:]]) for i in range(labels_srt.shape[0]) ]

    # Convert to numpy arrays
    cnt_chg_combi = np.vstack(cnt_chg_combi).T

    # Apply index to counts
    cnt_chg_combi_srt = cnt_chg_combi[:, srt_idx_combi]
else:
    # Create empty if no data
    cnt_chgs_pre_srt = np.repeat(np.nan, labels_srt.shape[0]).reshape((1, labels_srt.shape[0]))
    cnt_chgs_dur_srt = np.repeat(np.nan, labels_srt.shape[0]).reshape((1, labels_srt.shape[0]))

# Convert to numpy array
combi_labels = np.vstack(combi_labels)
# Apply index to labels
combi_labels_srt = combi_labels[srt_idx_combi, :].astype("int")
# Create a list with the signal changes as text
signal_changes = [ (type_dict[combi_labels_srt[i,0]], type_dict[combi_labels_srt[i,1]]) for i in range(0, combi_labels_srt.shape[0], 1) ]


# If requested export the data to test them later
if export_test_data:
    
    # Create an index stating during and previous to stimulation
    section_idx = np.concatenate([np.repeat("pre", cnt_chgs_pre_srt_mf.shape[0]), np.repeat("dur", cnt_chgs_dur_srt_mf.shape[0]), 
                              np.repeat("pre", cnt_chgs_pre_srt.shape[0]), np.repeat("dur", cnt_chgs_dur_srt.shape[0])])
    # Second index for the condition (m/mf)
    condition_idx = np.concatenate([np.repeat("mf", cnt_chgs_pre_srt_mf.shape[0]), np.repeat("mf", cnt_chgs_dur_srt_mf.shape[0]),
                                    np.repeat("m", cnt_chgs_pre_srt.shape[0]), np.repeat("m", cnt_chgs_dur_srt.shape[0])])
    # Stack together all data
    change_data_combined = np.vstack([cnt_chgs_pre_srt_mf, cnt_chgs_dur_srt_mf,
                                    cnt_chgs_pre_srt, cnt_chgs_dur_srt])
    # Create labels for the columns (singal switches in text)
    col_names = [ f"{type_dict[labels_srt[i,0]]}_{type_dict[labels_srt[i,1]]}" for i in range(0, labels_srt.shape[0]) ]

    # Combine all data and indices in a data frame
    change_data_testing = pd.DataFrame(change_data_combined, index=section_idx, columns=col_names)
    change_data_testing["condition"] = condition_idx
    # Sort the data to be in the same order as the combined data above
    change_data_testing = change_data_testing.iloc[:, [3,5,6, 7,8,10, 11,12,13, 0,1,2, 15]]
    # Export the data
    change_data_testing.to_csv(f"../additional_files/{exp_grp}_all_changes.csv")

#### Export the data

In [10]:
# Open a data file for writing
fdat = h5py.File(f"../additional_files/{exp_grp}_results.hdf5", "w")

try:
    if mf_cond:
        # Store the counts of all possible changes
        fall_changes_mf = fdat.create_dataset("/male_female/all_changes_cntd", data=cnt_chg_combi_srt_mf)
        # Label the dimensions
        fall_changes_mf.dims[0].label = "experiments"
        fall_changes_mf.dims[1].label = "signal type changes"
        # Save an index and a description
        fall_changes_mf.attrs.create("index", signal_changes)
        fall_changes_mf.attrs.create("description", """All possible signal type changes counted and normalized to #/(trial*s). 
        Entries are paired (0 and 1, 2 and 3, ...). Even numbers represent counts before stimulation, 
        odd numbers during stimulation.""")

        # Store the global changes in signal amounts, include index and description
        fglobal_changes_mf = fdat.create_dataset("/male_female/global_signal_diff", data=global_signal_changes_mf)
        fglobal_changes_mf.attrs.create("index", global_changes_idx)
        fglobal_changes_mf.attrs.create("description", """"Changes for all signal types upon activation measured as events 
        or s per minute. Changes are averaged over all stimulus intensities.""")

        # Store the changes in signal amounts separted by stimulus intensity
        ffi_counts_mf = fdat.create_dataset("/male_female/fi_signal_diff", data=all_counts_mf)
        ffi_counts_mf.attrs.create("signal_index", np.array(signal_index, dtype="S"))
        ffi_counts_mf.attrs.create("stimulus_index", stim_index)
        ffi_counts_mf.attrs.create("descriperion", """Changes for all signal types upon activation separately for each 
        stimulus intensity. Measured as events or s per minute. Changes are averaged for 
        multiple occurrances of the same stimulus intensity.""")

        # Store the raster matirces and label the dimensions
        fraster_p_mf = fdat.create_dataset("/male_female/raster_mat_pulse", data=r_mat_p_mf)
        fraster_p_mf.dims[0].label = "trials"
        fraster_p_mf.dims[1].label = "time"
        fraster_s_mf = fdat.create_dataset("/male_female/raster_mat_sine", data=r_mat_s_mf)
        fraster_s_mf.dims[0].label = "trials"
        fraster_s_mf.dims[1].label = "time"
        fraster_v_mf = fdat.create_dataset("/male_female/raster_mat_vibration", data=r_mat_v_mf)
        fraster_v_mf.dims[0].label = "trials"
        fraster_v_mf.dims[1].label = "time"
        # Save descriptions
        fraster_p_mf.attrs.create("description", """Raster matrix containing 1 at each position where pulses were found. 
        Pulse trains are stored in a continuous way, i.e. there is no information about 
        the number of pulses. Trials are stored along axis 0, time along axis 1. Trials 
        are sorted by animal.""")
        fraster_s_mf.attrs.create("description", """Raster matrix containing 2 at each position where sine was found. 
        Trials are stored along axis 0, time along axis 1. Trials are sorted by animal.""")
        fraster_v_mf.attrs.create("description", """Raster matrix containing 3 at each position where vibrations were 
        found. Vibration trains are stored in a continuous way, i.e. there is no information 
        about the number of vibrations. Trials are stored along axis 0, time along axis 1. 
        Trials are sorted by animal.""")
        # Store the corresponding time vector for the raster matrices
        fraster_time_mf = fdat.create_dataset("/male_female/raster_mat_time", data=raster_time)
        fraster_time_mf.attrs.create("description", """Time vector ([s]) corresponing to axis 1 of the raster matrices.""")
        # Store the stimulus values correspondin to the trials
        fraster_trials_mf = fdat.create_dataset("/male_female/raster_mat_stimuli", data=np.concatenate(stim_volts_mf))
        fraster_trials_mf.attrs.create("description", """Stimulus intensities (volt) for the trials of the raster matrices. 
        Corresponds to axis 0 of the raster matirces.""")

        # Store the overlaps between signal types
        foverlap_vp_mf = fdat.create_dataset("male_female/vp_overlap", data=overlaps_vp_mf)
        foverlap_vp_mf.attrs.create("trial_index", overlaps_vp_mf_idx)
        foverlap_vp_mf.attrs.create("description", """Potential overlaps between sine and pulse. The overlaps are stored 
        as pairs of two float numbers corresponding to start and stop values. The corresponding trials
        in which each overlap occurred are stored as an attribute.""")
        foverlap_vs_mf = fdat.create_dataset("male_female/vs_overlap", data=overlaps_vs_mf)
        foverlap_vs_mf.attrs.create("trial_index", overlaps_vs_mf_idx)
        foverlap_vs_mf.attrs.create("description", """Potential overlaps between sine and vibration. The overlaps are stored 
        as pairs of two float numbers corresponding to start and stop values. The corresponding trials
        in which each overlap occurred are stored as an attribute.""")


    if sm_cond:
        # Store the counts of all possible changes
        fall_changes = fdat.create_dataset("/male/all_changes_cntd", data=cnt_chg_combi_srt)
        fall_changes.attrs.create("index", signal_changes)
        fall_changes.attrs.create("description", """All possible signal type changes counted and normalized to #/(trial*s). 
        Entries are paired (0 and 1, 2 and 3, ...). Even numbers represent counts before stimulation, 
        odd numbers during stimulation.""")

    
        # Store the global changes in signal amounts, include index and description
        fglobal_changes = fdat.create_dataset("/male/global_signal_diff", data=global_signal_changes)
        fglobal_changes.attrs.create("index", global_changes_idx)
        fglobal_changes.attrs.create("description", """"Changes for all signal types upon activation measured as events or 
        s per minute. Changes are averaged over all stimulus intensities.""")

    
        # Store the changes in signal amounts separted by stimulus intensity
        ffi_counts = fdat.create_dataset("/male/fi_signal_diff", data=all_counts)
        ffi_counts.attrs.create("signal_index", np.array(signal_index, dtype="S"))
        ffi_counts.attrs.create("stimulus_index", stim_index)
        ffi_counts.attrs.create("descriperion", """Changes for all signal types upon activation separately for each 
        stimulus intensity. Measured as events or s per minute. Changes are averaged for 
        multiple occurrances of the same stimulus intensity.""")


        # Store the raster matirces and label the dimensions
        fraster_p = fdat.create_dataset("/male/raster_mat_pulse", data=r_mat_p)
        fraster_p.dims[0].label = "trials"
        fraster_p.dims[1].label = "time"
        fraster_s = fdat.create_dataset("/male/raster_mat_sine", data=r_mat_s)
        fraster_s.dims[0].label = "trials"
        fraster_s.dims[1].label = "time"
        fraster_v = fdat.create_dataset("/male/raster_mat_vibration", data=r_mat_v)
        fraster_v.dims[0].label = "trials"
        fraster_v.dims[1].label = "time"
        fraster_p.attrs.create("description", """Raster matrix containing 1 at each position where pulses were found. 
        Pulse trains are stored in a continuous way, i.e. there is no information about the 
        number of pulses. Trials are stored along axis 0, time along axis 1. Trials are 
        sorted by animal.""")
        fraster_s.attrs.create("description", """Raster matrix containing 2 at each position where sine was found. 
        Trials are stored along axis 0, time along axis 1. Trials are sorted by animal.""")
        fraster_v.attrs.create("description", """Raster matrix containing 3 at each position where vibrations were 
        found. Vibration trains are stored in a continuous way, i.e. there is no information 
        about the number of vibrations. Trials are stored along axis 0, time along axis 1. 
        Trials are sorted by animal.""")
        # Store the time vector
        fraster_time = fdat.create_dataset("male/raster_mat_time", data=raster_time)
        fraster_time.attrs.create("description", """Time vector ([s]) corresponing to axis 1 of the raster matrices.""")
        # Store the stimulus values correspondin to the trials
        fraster_trials = fdat.create_dataset("/male/raster_mat_stimuli", data=np.concatenate(stim_volts))
        fraster_trials.attrs.create("description", """Stimulus intensities (volt) for the trials of the raster matrices. 
        Corresponds to axis 0 of the raster matirces.""")

        # Store the overlaps between signal types
        foverlap_vp = fdat.create_dataset("male/vp_overlap", data=overlaps_vp)
        foverlap_vp.attrs.create("trial_index", overlaps_vp_idx)
        foverlap_vp.attrs.create("description", """Potential overlaps between sine and pulse. The overlaps are stored 
        as pairs of two float numbers corresponding to start and stop values. The corresponding trials
        in which each overlap occurred are stored as an attribute.""")
        foverlap_vs = fdat.create_dataset("male/vs_overlap", data=overlaps_vs)
        foverlap_vs.attrs.create("trial_index", overlaps_vs_idx)
        foverlap_vs.attrs.create("description", """Potential overlaps between sine and vibration. The overlaps are stored 
        as pairs of two float numbers corresponding to start and stop values. The corresponding trials
        in which each overlap occurred are stored as an attribute.""")

finally:
    # Close the data file
    fdat.close()